In [1]:
from PIL import Image
import h5py
import anubisPlotUtils as anPlot
import json
import numpy as np
import os
import hist as hi
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
matplotlib.use('TkAgg')  # or 'Qt5Agg', 'GTK3Agg', etc.
import mplhep as hep
hep.style.use([hep.style.ATLAS])
import sys
import AnalysisToolAnubis as AT
import pandas as pd
import tkinter as tk
from tkinter import ttk
from pandasgui import show
from matplotlib.patches import Circle
from matplotlib.backends.backend_pdf import PdfPages
from scipy.signal import convolve2d
from scipy.ndimage import label, find_objects
from scipy.spatial.distance import pdist, squareform
from matplotlib.backends.backend_pdf import PdfPages
import ipywidgets as widgets
from IPython.display import display


ERROR:root:Cannot activate multiple GUI eventloops


In [2]:
#Importing the data
thisData = AT.importDatafile('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubisData\\60sRun_24_3_4.h5')


In [5]:
mapping = {
    0: {
        (0, 31): ('rpc1', 'eta'),
        (32, 63): ('rpc1', 'phi1'),
        (64, 95): ('rpc1', 'phi2'),
        (96, 127): ('rpc2', 'eta'),
    },
    1: {
        (0, 31): ('rpc2', 'phi1'),
        (32, 63): ('rpc2', 'phi2'),
        (64, 95): ('rpc3', 'eta'),
        (96, 127): ('rpc3', 'phi1'),
    },
    2: {
        (0, 31): ('rpc3', 'phi2'),
        (32, 63): ('rpc4', 'eta'),
        (64, 95): ('rpc4', 'phi1'),
        (96, 127): ('rpc4', 'phi2'),
    },
    3: {
        (0, 31): ('rpc5', 'eta'),
        (32, 63): ('rpc5', 'phi1'),
        (64, 95): ('rpc5', 'phi2'),
        (96, 127): ('rpc6', 'eta'),
    },
    4: {
        (0, 31): ('rpc6', 'phi1'),
        (32, 63): ('rpc6', 'phi2'),
    },
}

In [6]:
def assign_to_coordinates(tdc, channel, mapping):
    for channel_range, (rpc, side) in mapping.get(tdc, {}).items():
        if channel_range[0] <= channel <= channel_range[1]:
            offset = channel - channel_range[0]
            if side == 'eta':
                return [rpc, 'eta', [0, offset]]
            elif side in ['phi1', 'phi2']:
                return [rpc, 'phi', [offset, 0]]
    return ['Null', 'Null', [tdc, channel]]  # If no match found
def remake_data(Data, mapping):
    full_data = []
    for tdc in range(0, len(Data)):
        for events in range(0, len(Data[tdc])): 
            for hits in Data[tdc][events]:
                channel = (hits>>24)&0x7f
                coord = assign_to_coordinates(tdc, channel, mapping)
                event_time = hits&0xfffff
                
            
                
                full_data.append({
                    'tdc': tdc,
                    'event number': events,
                    'channel':channel,
                    'rpc number': coord[0],
                    'strip direction': coord[1],
                    'hit coordinate': coord[2],
                    'time': event_time
                })
    df = pd.DataFrame(full_data)
        
    return df
    
    

In [7]:
df = remake_data(thisData, mapping)


In [53]:
#slightly different way to store data, but does the same thing, it goes like rpc - event - strip
etaHits, phiHits = AT.divideHitCountsByRPC(thisData)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [54]:
def calculate_cluster_metrics(event, minHit=0, maxHit=20):
    cluster_sizes = []
    cluster_locations = []
    current_cluster_size = 0
    start_index = None
    
    for index, i in enumerate(event):
        if i != 0:
            current_cluster_size += i
            if start_index is None:
                start_index = index  
        elif i == 0 and current_cluster_size >= minHit and current_cluster_size < maxHit:
            if start_index is not None:           
                cluster_midpoint = (start_index + index) / 2.0 
                cluster_locations.append(cluster_midpoint)
                cluster_sizes.append(current_cluster_size)
                current_cluster_size = 0
                start_index = None 
        elif i == 0:
            current_cluster_size = 0
            start_index = None 
            
    if current_cluster_size >= minHit and current_cluster_size < maxHit and start_index is not None:
        cluster_midpoint = (start_index + len(event)) / 2.0 
        cluster_locations.append(cluster_midpoint)
        cluster_sizes.append(current_cluster_size)
    
    return cluster_locations, cluster_sizes


In [55]:
columns = ['RPC Number', 'Event Number', 'cluster_locations', 'Cluster_size']
dfcluseta = pd.DataFrame(columns=columns)
dfclusphi = pd.DataFrame(columns=columns)

for i, rpc_data in enumerate(etaHits):
    for j, event in enumerate(rpc_data):
        cluster_locations, cluster_sizes = calculate_cluster_metrics(event)
        for _ in range(len(cluster_sizes)):
            
            dfcluseta = dfcluseta.append({'RPC Number': int(i), 'Event Number': int(j), 'cluster_locations': cluster_locations[_], 'Cluster_size': int(cluster_sizes[_])}, ignore_index=True)

for i, rpc_data in enumerate(phiHits):
    for j, event in enumerate(rpc_data):
        cluster_locations, cluster_sizes = calculate_cluster_metrics(event)
        filtered_pairs = [(x, y) for x, y in zip(cluster_locations, cluster_sizes) if x >= 1]

        if filtered_pairs:
            filtered_my_list, filtered_list2 = zip(*filtered_pairs)

            cluster_locations = list(filtered_my_list)
            cluster_sizes = list(filtered_list2)
        else:

            cluster_locations = []
            cluster_sizes = []
        for _ in range(len(cluster_sizes)):
            
            dfclusphi = dfclusphi.append({'RPC Number': int(i), 'Event Number': int(j), 'cluster_locations': cluster_locations[_], 'Cluster_size': int(cluster_sizes[_])}, ignore_index=True)

C:\Users\Peter\AppData\Local\Temp\ipykernel_19680\695374553.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfcluseta = dfcluseta.append({'RPC Number': int(i), 'Event Number': int(j), 'cluster_locations': cluster_locations[_], 'Cluster_size': int(cluster_sizes[_])}, ignore_index=True)
C:\Users\Peter\AppData\Local\Temp\ipykernel_19680\695374553.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfcluseta = dfcluseta.append({'RPC Number': int(i), 'Event Number': int(j), 'cluster_locations': cluster_locations[_], 'Cluster_size': int(cluster_sizes[_])}, ignore_index=True)
C:\Users\Peter\AppData\Local\Temp\ipykernel_19680\695374553.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfcluseta = dfcluseta.append({'RPC Num

In [45]:
print(calculate_cluster_metrics(phiHits[0][0]))

([52.5], [5])


In [56]:
rpc_dfseta = {rpc: dfcluseta[dfcluseta['RPC Number'] == rpc].reset_index(drop=True) for rpc in dfcluseta['RPC Number'].unique()}
rpc_dfsphi = {rpc: dfclusphi[dfclusphi['RPC Number'] == rpc].reset_index(drop=True) for rpc in dfclusphi['RPC Number'].unique()}

def plot_rpc_histograms(rpc_dfseta, rpc_dfsphi):
    with PdfPages('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\rpc_cluster_analysisaLL.pdf') as pdf:
        for rpc in rpc_dfseta.keys():
            df_eta = rpc_dfseta[rpc]
            df_phi = rpc_dfsphi[rpc]
            for df, dataset_name in zip([df_eta, df_phi], ['eta', 'phi']):
                all_cluster_locations = []
                all_cluster_sizes = []
                
                for _, row in df.iterrows():
                    locs = row['cluster_locations']
                    sizes = row['Cluster_size']
                    
                    if not isinstance(locs, list):
                        locs = [locs] if not pd.isnull(locs) else []
                    if not isinstance(sizes, list):
                        sizes = [sizes] if not pd.isnull(sizes) else []
                    
                    all_cluster_locations.extend(locs)
                    all_cluster_sizes.extend(sizes)
                
                locations_series = pd.Series(all_cluster_locations)
                sizes_series = pd.Series(all_cluster_sizes)
                
                location_frequencies = locations_series.value_counts().sort_index()
                size_frequencies = sizes_series.value_counts().sort_index()
                
                fig, axs = plt.subplots(1, 2, figsize=(12, 6))
                fig.suptitle(f'RPC {rpc} - {dataset_name.upper()}')

                if not location_frequencies.empty:
                    axs[0].bar(location_frequencies.index, location_frequencies.values, color='skyblue', edgecolor='black')
                axs[0].set_title(f'Cluster Locations')
                axs[0].set_xlabel('Cluster Location')
                axs[0].set_ylabel('Frequency')

                if not size_frequencies.empty:
                    axs[1].bar(size_frequencies.index, size_frequencies.values, color='salmon', edgecolor='black')
                axs[1].set_title(f'Cluster Sizes')
                axs[1].set_xlabel('Cluster Size')
                axs[1].set_ylabel('Frequency')
                
                if dataset_name == 'phi':
                    axs[0].set_xlim(0, 64)
                    # axs[1].set_xlim(0, 64)
                if dataset_name == 'eta':
                    axs[0].set_xlim(0, 32)
                    # axs[1].set_xlim(0, 32)
                    
                plt.tight_layout(rect=[0, 0.03, 1, 0.95])
                pdf.savefig(fig)
                plt.close()


In [56]:
plot_rpc_histograms(rpc_dfseta, rpc_dfsphi)

In [7]:
etaTime, phiTime = AT.divideEventTimesByRPC(thisData)

In [14]:
show(df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


It would be easier to work straight from the data frame
first, we want to test how does dark counts work, by only choosing hits that are larger than 300ns

In [89]:
df_filtered300plus = df[df['time'] <= 245]

In [76]:
show(df_filtered300plus)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [90]:
def dataframe_to_thisData(df):
    max_tdc = df['tdc'].max()
    max_event_number = df['event number'].max()
    
    reconstructed_data = [[[] for _ in range(max_event_number + 1)] for _ in range(max_tdc + 1)]

    for index, row in df.iterrows():
        tdc = row['tdc']
        event_number = row['event number']
        channel = row['channel']
        time = row['time']
        hits = (channel << 24) | time
        reconstructed_data[tdc][event_number].append(hits)

    return reconstructed_data

In [91]:
thisDat300plus = dataframe_to_thisData(df_filtered300plus)

In [92]:
etaHits300plus, phiHits300plus = AT.divideHitCountsByRPC(thisDat300plus)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [93]:
columns = ['RPC Number', 'Event Number', 'cluster_locations', 'Cluster_size']
dfcluseta300plus = pd.DataFrame(columns=columns)
dfclusphi300plus = pd.DataFrame(columns=columns)

for i, rpc_data in enumerate(etaHits300plus):
    for j, event in enumerate(rpc_data):
        cluster_locations, cluster_sizes = calculate_cluster_metrics(event)
        for _ in range(len(cluster_sizes)):
            
            dfcluseta300plus = dfcluseta300plus.append({'RPC Number': int(i), 'Event Number': int(j), 'cluster_locations': cluster_locations[_], 'Cluster_size': int(cluster_sizes[_])}, ignore_index=True)

for i, rpc_data in enumerate(phiHits300plus):
    for j, event in enumerate(rpc_data):
        cluster_locations, cluster_sizes = calculate_cluster_metrics(event)
        # filtered_pairs = [(x, y) for x, y in zip(cluster_locations, cluster_sizes) if x >= 1]

        # if filtered_pairs:
        #     filtered_my_list, filtered_list2 = zip(*filtered_pairs)

        #     cluster_locations = list(filtered_my_list)
        #     cluster_sizes = list(filtered_list2)
        # else:

        #     cluster_locations = []
        #     cluster_sizes = []
        for _ in range(len(cluster_sizes)):
            
            dfclusphi300plus = dfclusphi300plus.append({'RPC Number': int(i), 'Event Number': int(j), 'cluster_locations': cluster_locations[_], 'Cluster_size': int(cluster_sizes[_])}, ignore_index=True)

C:\Users\Peter\AppData\Local\Temp\ipykernel_19680\3272357497.py:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Peter\AppData\Local\Temp\ipykernel_19680\3272357497.py:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Peter\AppData\Local\Temp\ipykernel_19680\3272357497.py:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Peter\AppData\Local\Temp\ipykernel_19680\3272357497.py:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Peter\AppData\Local\Temp\ipykernel_19680\3272357497.py:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

In [94]:
rpc_dfseta300plus = {rpc: dfcluseta300plus[dfcluseta300plus['RPC Number'] == rpc].reset_index(drop=True) for rpc in dfcluseta300plus['RPC Number'].unique()}
rpc_dfsphi300plus = {rpc: dfclusphi300plus[dfclusphi300plus['RPC Number'] == rpc].reset_index(drop=True) for rpc in dfclusphi300plus['RPC Number'].unique()}
plot_rpc_histograms(rpc_dfseta300plus, rpc_dfsphi300plus)

There wasn't a clear cutoff, but it is clear that below 240 and above 300 we are getting a LOT of single hits, these are ofcourse dark hits.
we should really ignore anything above 10, since that probably means when all the strips fired off at the same time

We can arbitrarily define a timing window of lets say 20 nanoseconds for each event to be considered independent to each other, this will allow me to generate a series of coincidence, and actually finding the coincidences correctly rather than arbitrarily...

In [102]:
show(df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [135]:
def find_coincidences(group):
    coincidences = []
    for i in range(len(group)):
        for j in range(i + 1, len(group)):

            time_i = group.iloc[i]['time']
            time_j = group.iloc[j]['time']
            coord_i = group.iloc[i]['hit coordinate']
            coord_j = group.iloc[j]['hit coordinate']
            

            if (coord_i[0] != 0 and coord_j[1] != 0 and coord_i[1] == 0 and coord_j[0] == 0) or \
               (coord_i[1] != 0 and coord_j[0] != 0 and coord_i[0] == 0 and coord_j[1] == 0):

                if abs(time_i - time_j) < 10:

                    average_time = (time_i + time_j) / 2

                    coincidence_coord = [max(coord_i[0], coord_j[0]), max(coord_i[1], coord_j[1])]
                    
                    coincidences.append({
                        'event number': group.iloc[i]['event number'],
                        'rpc number': group.iloc[i]['rpc number'],
                        'coincidence coordinate': coincidence_coord,
                        'coincidence time': average_time
                    })
    return coincidences

In [129]:
print(df.groupby(['event number', 'rpc number']))

In [132]:
coincidences = []
for _, group in df.groupby(['event number', 'rpc number']):
    coincidences.extend(find_coincidences(group))

coincidences_df = pd.DataFrame(coincidences)

In [133]:
show(coincidences_df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [134]:
show(df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


Before I actually do this, I actually wanted to do something more on 1D with timing window first.... I will do that tmr